In [7]:
# 这是最简单的 LTSM 模型
import torch
import torch.nn as nn

# 初始化的 lstm 模型
lstm_model = nn.LSTM(10, 20, 2)                    # 参数为(input_size, hidden_size, num_layers)

# 模型的cell接受的输入参数
x = torch.randn(5, 3, 10)                         # 参数为(seq_len, batch, input_size)
# 模型的cell接受的输入hidden_state；也是前一个cell的输出hidden_state
h0 = torch.randn(2, 3, 20)                         # 参数为(num_layers * num_directions, batch, hidden_size)
# 模型的cell初始值
c0 = torch.randn(2, 3, 20)                         # 参数为(num_layers * num_directions, batch, hidden_size)

y, (h1, c1) = lstm_model(x, (h0, c0))

print(y.size())
print(h1.size())
print(c1.size())


torch.Size([5, 3, 20])
torch.Size([2, 3, 20])
torch.Size([2, 3, 20])


In [ ]:
# 这是用LTSM模型做 MNIST 图片的识别
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt